In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
import matplotlib.pyplot as plt # For potential plotting

In [2]:
dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')
birmingham_disposal = pd.read_excel(r'../../data/raw/birm_Disposal( in period 09052025).xlsx')
#ladataset = pd.read_excel(r'../../data/raw/Local_government_finance_data_download.xlsx',sheet_name='Spending power totals')
#housing_net_supply_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Net supply')
#housing_stock_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Housing stock')
#population = pd.read_excel('../../data/raw/population_by_age.xlsx', sheet_name='Age bands')

In [3]:
# Section 114 notice dates by local authority (first relevant notice for each)
s114_dates = {
    'Nottingham': '2021-12-15',
    'Birmingham': '2023-09-05',
    'Woking': '2023-06-07',
    'Thurrock': '2022-12-18',
    'Croydon': '2020-11-11',
    'Slough': '2020-11-11',
    'North Northamptonshire': '2018-02-02',
    'West Northamptonshire': '2018-02-02',
}

# Convert to datetime
s114_dates = {k: pd.to_datetime(v) for k, v in s114_dates.items()}

# Map treatment date based on local authority
dataset['treatment_date'] = dataset['local_authority'].map(s114_dates)

# Create binary treatment indicator
dataset['treatment'] = dataset['treatment_date'].notna().astype(int)

In [ ]:
df_bham_charities = dataset[dataset['local_authority'] == 'Birmingham'].copy()
s114_date = pd.to_datetime('2023-09-05') # Birmingham Section 114 notice date
end_of_study_date = pd.to_datetime('2025-03-31') # Your actual end date
df_bham_charities['date_of_registration'] = pd.to_datetime(df_bham_charities['date_of_registration'], errors='coerce')
df_bham_charities['date_of_removal'] = pd.to_datetime(df_bham_charities['date_of_removal'], errors='coerce')

df_bham_charities['is_removed_by_end_of_study'] = (
    (df_bham_charities['date_of_removal'].notna()) &
    (df_bham_charities['date_of_removal'] <= end_of_study_date)
).astype(int)

print(f"Number of charities considered removed by {end_of_study_date}: {df_bham_charities['is_removed_by_end_of_study'].sum()}")
print(f"Total Birmingham charities in scope: {len(df_bham_charities)}")

# Section 114 flag at end_of_study_date
df_bham_charities['section_114_flag_at_end_of_study'] = int(s114_date <= end_of_study_date)

# Cumulative disposals up to end_of_study_date
birmingham_disposal['DISPLDATE'] = pd.to_datetime(birmingham_disposal['DISPLDATE'])

# Filter disposals up to the end_of_study_date
relevant_disposals_at_end = birmingham_disposal[
    (birmingham_disposal['DISPLDATE'] <= end_of_study_date)
].copy()


# --- 6. Select final features for the model ---
# Drop columns that were problematic (zero variance) or are no longer needed
final_df_for_logistic = df_bham_charities.copy()

Number of charities considered removed by 2025-03-31 00:00:00: 619
Total Birmingham charities in scope: 3085


In [21]:
dataset['Other'].sum()

np.int64(88033)

In [16]:
model_covariates = [
    'section_114_flag_at_end_of_study',
    'charity_has_land',
]

# --- 7. Prepare X (features) and y (target) ---
X = final_df_for_logistic[model_covariates]
y = final_df_for_logistic['is_removed_by_end_of_study']

# Drop rows with NaNs in X or y
initial_rows = len(X)
data_for_model = pd.concat([X, y], axis=1).dropna()
X_clean = data_for_model[model_covariates]
y_clean = data_for_model['is_removed_by_end_of_study']
print(f"Dropped {initial_rows - len(X_clean)} rows due to NaNs in model features or target.")

# Add a constant (intercept) to the features for statsmodels
X_clean = sm.add_constant(X_clean, prepend=False) # prepend=False puts it at the end

print(f"\n--- Data for Logistic Regression (First 5 rows) ---")
print(pd.concat([X_clean, y_clean], axis=1).head())
print(f"Number of observations for Logistic Regression: {len(X_clean)}")
print(f"Number of 'removed' outcomes: {y_clean.sum()}")
print(f"Number of 'not removed' outcomes: {(y_clean == 0).sum()}")

# Convert 'True'/'False' strings or bools to 1/0
X_clean['charity_has_land'] = X_clean['charity_has_land'].astype(bool).astype(int)

print(X_clean.dtypes)
print(y_clean.dtype)

# --- 8. Fit the Logistic Regression Model ---
# Use GLM with Binomial family for Logistic Regression
model = GLM(y_clean, X_clean, family=families.Binomial())
logistic_results = model.fit()

print("\n--- Logistic Regression Model Summary ---")
logistic_results.summary()

Dropped 411 rows due to NaNs in model features or target.

--- Data for Logistic Regression (First 5 rows) ---
     section_114_flag_at_end_of_study charity_has_land  \
353                                 1             True   
494                                 1            False   
596                                 1            False   
690                                 1             True   
709                                 1            False   

     is_removed_by_end_of_study  
353                           0  
494                           1  
596                           0  
690                           0  
709                           0  
Number of observations for Logistic Regression: 2674
Number of 'removed' outcomes: 342
Number of 'not removed' outcomes: 2332
section_114_flag_at_end_of_study    int64
charity_has_land                    int64
dtype: object
int64

--- Logistic Regression Model Summary ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                      
======================================================================================
Dep. Variable:     is_removed_by_end_of_study   No. Observations:                 2674
Model:                                    GLM   Df Residuals:                     2672
Model Family:                        Binomial   Df Model:                            1
Link Function:                          Logit   Scale:                          1.0000
Method:                                  IRLS   Log-Likelihood:                -1011.7
Date:                        Thu, 03 Jul 2025   Deviance:                       2023.5
Time:                                22:09:25   Pearson chi2:                 2.67e+03
No. Iterations:                             5   Pseudo R-squ. (CS):           0.007988
Covariance Type:                    nonrobust                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
section_114_flag_at_end_of_study    -1.7511      0.066    -26.463      0.000      -1.881      -1.621
charity_has_land                    -0.6143      0.138     -4.438      0.000      -0.886      -0.343
====================================================================================================
"""

In [ ]:
dataset.columns

In [ ]:
birmingham_disposal.columns

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
import matplotlib.pyplot as plt # For potential plotting
import warnings

# --- 4. Prepare static covariates ---
# Log transform income, handle potential 0s or NaNs gracefully
df_bham_charities['charity_income_log'] = np.log(df_bham_charities['latest_income'].replace(0, 1) + 1) # Add 1 to avoid log(0) and handle 0 incomes
#df_bham_charities['charity_has_land'] = df_bham_charities['charity_has_land'].astype(int) # Ensure integer type

# Convert charity_type to dummy variables
# statsmodels formula interface handles this automatically, but explicitly is good for control
df_bham_charities = pd.get_dummies(df_bham_charities, columns=['charity_type'], prefix='charity_type', drop_first=True) # drop_first avoids multicollinearity

# Note: Activity flags are already binary (0/1) from your data.

# --- 5. Prepare snapshot time-varying covariates ---

# Section 114 flag at end_of_study_date
df_bham_charities['section_114_flag_at_end_of_study'] = int(s114_date <= end_of_study_date)

# Cumulative disposals up to end_of_study_date
birmingham_disposal['DISPLDATE'] = pd.to_datetime(birmingham_disposal['DISPLDATE'])

# Filter disposals up to the end_of_study_date
relevant_disposals_at_end = birmingham_disposal[
    (birmingham_disposal['DISPLDATE'] <= end_of_study_date)
].copy()

# Calculate cumulative sums for these disposals
# Add a small value to DEEDCONS before log transform to handle potential 0s if they existed
relevant_disposals_at_end['DEEDCONS_log'] = np.log(relevant_disposals_at_end['DEEDCONS'] + 0.01)

# Sum up the cumulative values at the end of the study. Since this is for a static model,
# each charity gets the same total sum of disposals *up to that point in time*.
# If you wanted to relate disposals to specific charities (e.g., if you had property links),
# this would be more complex. For now, it's a general "state of disposals in Birmingham."
cumulative_bham_disposals_log_value_at_end = relevant_disposals_at_end['DEEDCONS_log'].sum()
cumulative_disposal_type_community_centre_at_end = (relevant_disposals_at_end['TYPEPROP'] == 'Community Centre').sum()
cumulative_disposal_type_leisure_facility_at_end = (relevant_disposals_at_end['TYPEPROP'] == 'Leisure Facility').sum()
cumulative_disposal_type_office_at_end = (relevant_disposals_at_end['TYPEPROP'] == 'Office').sum()
cumulative_disposal_type_land_at_end = (relevant_disposals_at_end['TYPEPROP'] == 'Land').sum()
cumulative_disposal_method_community_transfer_at_end = (relevant_disposals_at_end['TYPECONSD'] == 'Community Transfer').sum()


# Assign these as new columns to your main charities DataFrame
df_bham_charities['cumulative_bham_disposals_log_value_at_end'] = cumulative_bham_disposals_log_value_at_end
df_bham_charities['cumulative_disposal_type_community_centre_at_end'] = cumulative_disposal_type_community_centre_at_end
df_bham_charities['cumulative_disposal_type_leisure_facility_at_end'] = cumulative_disposal_type_leisure_facility_at_end
df_bham_charities['cumulative_disposal_type_office_at_end'] = cumulative_disposal_type_office_at_end
df_bham_charities['cumulative_disposal_type_land_at_end'] = cumulative_disposal_type_land_at_end
df_bham_charities['cumulative_disposal_method_community_transfer_at_end'] = cumulative_disposal_method_community_transfer_at_end


# --- 6. Select final features for the model ---
# Drop columns that were problematic (zero variance) or are no longer needed
final_df_for_logistic = df_bham_charities.copy()

# List of all covariates we want to include
# Note: Removed the activity flags that were all 0s in your DUMMY data.
# If your real data for these has variance, you can add them back.
# 'Grants_Related', 'Housing_Infrastructure', 'Human_Rights_Advocacy',
# 'Other', 'Other_Miscellaneous', 'Overseas_Famine_Relief', 'Religion_Faith'
# 'Children_Youth' also had low variance in dummy data, but check your real data for it.
# For now, let's stick to the common ones:
model_covariates = [
    'section_114_flag_at_end_of_study',
    'cumulative_bham_disposals_log_value_at_end',
    'cumulative_disposal_type_community_centre_at_end',
    'cumulative_disposal_type_leisure_facility_at_end',
    'cumulative_disposal_type_office_at_end',
    'cumulative_disposal_type_land_at_end',
    'cumulative_disposal_method_community_transfer_at_end',
    'charity_income_log',
    'charity_has_land',
    # Add other activity flags if they have sufficient variance in your real data
]

# Check for existence of columns and drop if they don't exist (e.g., if get_dummies didn't create them)
# This loop also removes the problematic zero-variance ones *if* they still exist
# and are problematic in the real data.
actual_model_covariates = []
for col in model_covariates:
    if col in final_df_for_logistic.columns:
        # Check variance before adding to final list
        if final_df_for_logistic[col].nunique() > 1: # Must have more than one unique value to vary
            actual_model_covariates.append(col)
        else:
            print(f"Skipping '{col}' due to zero or very low variance.")
    else:
        print(f"Warning: '{col}' not found in DataFrame. Check column creation or spelling.")

# --- 7. Prepare X (features) and y (target) ---
X = final_df_for_logistic[actual_model_covariates]
y = final_df_for_logistic['is_removed_by_end_of_study']

# Drop rows with NaNs in X or y
initial_rows = len(X)
data_for_model = pd.concat([X, y], axis=1).dropna()
X_clean = data_for_model[actual_model_covariates]
y_clean = data_for_model['is_removed_by_end_of_study']
print(f"Dropped {initial_rows - len(X_clean)} rows due to NaNs in model features or target.")

# Add a constant (intercept) to the features for statsmodels
X_clean = sm.add_constant(X_clean, prepend=False) # prepend=False puts it at the end

print(f"\n--- Data for Logistic Regression (First 5 rows) ---")
print(pd.concat([X_clean, y_clean], axis=1).head())
print(f"Number of observations for Logistic Regression: {len(X_clean)}")
print(f"Number of 'removed' outcomes: {y_clean.sum()}")
print(f"Number of 'not removed' outcomes: {(y_clean == 0).sum()}")

# --- 8. Fit the Logistic Regression Model ---
try:
    # Use GLM with Binomial family for Logistic Regression
    model = GLM(y_clean, X_clean, family=families.Binomial())
    logistic_results = model.fit()

    print("\n--- Logistic Regression Model Summary ---")
    print(logistic_results.summary())

    # --- Interpretation Tips ---
    print("\n--- Interpretation Notes ---")
    print("Coefficients (coef) are in log-odds. To get Odds Ratios (OR), exponentiate them (e.g., np.exp(coef)).")
    print("OR > 1 means an increase in the covariate is associated with higher odds of removal.")
    print("OR < 1 means an increase in the covariate is associated with lower odds of removal.")
    print("P>|z| (p-value) indicates statistical significance. Typically, < 0.05 is considered significant.")

except Exception as e:
    print(f"\n--- Error during Model Fitting ---")
    print(f"An error occurred: {e}")
    print("This could still be due to perfect collinearity or extremely sparse data in a specific categorical variable.")
    print("Review the 'Skipping ... due to zero or very low variance' messages above.")

In [ ]:
dataset['date_of_registration'] = pd.to_datetime(dataset['date_of_registration'])
dataset['date_of_removal'] = pd.to_datetime(dataset['date_of_removal'])
dataset['registration_month'] = dataset['date_of_registration'].dt.to_period('M')
dataset['removal_month'] = dataset['date_of_removal'].dt.to_period('M')

removed_by_month_la = (
    dataset
    .groupby(['local_authority', 'removal_month'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)
panel_df = removed_by_month_la.reset_index().melt(
    id_vars='local_authority',
    var_name='month',
    value_name='removed'
)

# Convert 'month' back to Period type for date calculations
panel_df['month'] = panel_df['month'].astype(str).apply(pd.Period)

# Add treatment info
treatment_map = dataset.groupby('local_authority')[['treatment', 'treatment_date']].first()
panel_df = panel_df.merge(treatment_map, on='local_authority', how='left')

# Compute 'post' indicator at monthly granularity
panel_df['treatment_month'] = panel_df['treatment_date'].dt.to_period('M')
panel_df['post'] = (panel_df['month'] >= panel_df['treatment_month']).astype(int)
panel_df.loc[panel_df['treatment'].isna(), 'post'] = 0  # untreated always 0

# Convert month Period to string for fixed effects
panel_df['month_str'] = panel_df['month'].astype(str)


In [ ]:
# Build formula with month fixed effects
formula = (
    'removed ~ post + treatment + treatment:post + '
    'C(local_authority) ' 
)

model = smf.ols(formula, data=panel_df).fit()

# Extract the coefficient table as a DataFrame
summary_df = model.summary2().tables[1]

# Filter for only the terms of interest
terms_to_show = ['post', 'treatment', 'treatment:post']
filtered_summary = summary_df.loc[summary_df.index.isin(terms_to_show)]

print(filtered_summary)

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# Sort by local authority and month
panel_df = panel_df.sort_values(['local_authority', 'month'])

# First differences
panel_df['D_removed'] = panel_df.groupby('local_authority')['removed'].diff()

# Treatment is constant per group; post varies over time, so we create an interaction
# This is the DiD estimator in differenced form
interaction = panel_df['post'] * panel_df['treatment']
panel_df['D_interaction'] = interaction.groupby(panel_df['local_authority']).diff()

# Drop rows with missing differences
panel_df_fd = panel_df.dropna(subset=['D_removed', 'D_interaction'])

# Run first-difference regression
fd_model = smf.ols('D_removed ~ D_interaction', data=panel_df_fd).fit()

# Print the result
print(fd_model.summary2().tables[1].loc[['D_interaction']])


In [ ]:
# Sort by local authority and month
panel_df = panel_df.sort_values(['local_authority', 'month'])

# Avoid log(0) by replacing 0 with a small constant (or drop)
panel_df['removed_adj'] = panel_df['removed'].replace(0, np.nan)

# Log transform the outcome
panel_df['log_removed'] = np.log(panel_df['removed_adj'])

# First difference of log outcome (i.e., log-change ≈ % change)
panel_df['D_log_removed'] = panel_df.groupby('local_authority')['log_removed'].diff()

# Interaction term: treatment x post, then differenced
interaction = panel_df['post'] * panel_df['treatment']
panel_df['D_interaction'] = interaction.groupby(panel_df['local_authority']).diff()

# Drop missing values
panel_df_fd = panel_df.dropna(subset=['D_log_removed', 'D_interaction'])

# Fit model
fd_model = smf.ols('D_log_removed ~ D_interaction', data=panel_df_fd).fit()

# Print result
print(fd_model.summary2().tables[1].loc[['D_interaction']])


In [ ]:
dataset['date_of_registration'] = pd.to_datetime(dataset['date_of_registration'])
dataset['date_of_removal'] = pd.to_datetime(dataset['date_of_removal'])
dataset['registration_month'] = dataset['date_of_registration'].dt.to_period('M')
dataset['removal_month'] = dataset['date_of_removal'].dt.to_period('M')

# Define full monthly range (from min registration to max removal month)
min_month = dataset['registration_month'].min()
max_month = dataset['removal_month'].max()
all_months = pd.period_range(min_month, max_month, freq='M')

# Compute number of active charities at the end of each month by local authority
active_by_month_la = {}

for month in all_months:
    # Active means registered before or on month AND not removed before or on month
    active_mask = (
        (dataset['registration_month'] <= month) &
        (
            dataset['removal_month'].isna() |
            (dataset['removal_month'] > month)
        )
    )
    active_count = (
        dataset[active_mask]
        .groupby('local_authority')
        .size()
        .rename(str(month))
    )
    active_by_month_la[month] = active_count

active_by_month_la_df = pd.DataFrame(active_by_month_la).fillna(0).astype(int)

# Count removals by month and local authority
removed_by_month_la = (
    dataset
    .groupby(['local_authority', 'removal_month'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Align indices of active and removal data
active_by_month_la_df = active_by_month_la_df.reindex(removed_by_month_la.index)

# Calculate removal rate per month
removal_rate_by_month_la = removed_by_month_la / active_by_month_la_df

# Convert wide to long format for panel data
panel_df = removal_rate_by_month_la.reset_index().melt(
    id_vars='local_authority',
    var_name='month',
    value_name='removed_rate'
)

# Convert 'month' back to Period type for date calculations
panel_df['month'] = panel_df['month'].astype(str).apply(pd.Period)

# Add treatment info
treatment_map = dataset.groupby('local_authority')[['treatment', 'treatment_date']].first()
panel_df = panel_df.merge(treatment_map, on='local_authority', how='left')

# Compute 'post' indicator at monthly granularity
panel_df['treatment_month'] = panel_df['treatment_date'].dt.to_period('M')
panel_df['post'] = (panel_df['month'] >= panel_df['treatment_month']).astype(int)
panel_df.loc[panel_df['treatment'].isna(), 'post'] = 0  # untreated always 0

# Convert month Period to string for fixed effects
panel_df['month_str'] = panel_df['month'].astype(str)

# Build formula with month fixed effects
formula = (
    'removed_rate ~ post + treatment + treatment:post + '
    'C(local_authority) ' 
)

# Fit model with clustered standard errors by local authority
model = smf.ols(formula, data=panel_df).fit()

summary_df = model.summary()

print(summary)


YEARLY

In [ ]:
# Ensure date columns are datetime
dataset['date_of_registration'] = pd.to_datetime(dataset['date_of_registration'])
dataset['date_of_removal'] = pd.to_datetime(dataset['date_of_removal'])

# Define FY end date mapping
# Get financial years from 2010 to 2025
fy_end_dates = (
    dataset['removal_fy']
    .dropna()
    .astype(int)
)
fy_end_dates = sorted(fy_end_dates[(fy_end_dates >= 2014) & (fy_end_dates <= 2025)].unique())

fy_end_date_map = {fy: pd.Timestamp(f"{int(fy)+1}-03-31") for fy in fy_end_dates}

# Compute number of active charities at end of each FY by council
active_by_fy_la = {}

for fy, end_date in fy_end_date_map.items():
    previous_end_date = end_date - pd.DateOffset(years=1)
    active_mask = (
        (dataset['date_of_registration'] <= previous_end_date) &
        (
            dataset['date_of_removal'].isna() |
            (dataset['date_of_removal'] > previous_end_date)
        )
    )
    active_count = (
        dataset[active_mask]
        .groupby('local_authority')
        .size()
        .rename(fy)
    )
    active_by_fy_la[fy] = active_count

active_by_fy_la_df = pd.DataFrame(active_by_fy_la).fillna(0).astype(int)

# Your original: removals by FY and local authority
removed_by_fy_la = (
    dataset
    .groupby(['local_authority', 'removal_fy'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Align indices
active_by_fy_la_df = active_by_fy_la_df.reindex(removed_by_fy_la.index)

# Calculate removal rate per FY
removal_rate_by_fy_la = removed_by_fy_la / active_by_fy_la_df
removal_rate_by_fy_la

In [ ]:
# Step 1: Wide to long
panel_df = removal_rate_by_fy_la.reset_index().melt(
    id_vars='local_authority',
    var_name='financial_year',
    value_name='removed_rate'
)

# Step 2: Add treatment info
# Map treatment and treatment_date from the original dataset
treatment_map = dataset.groupby('local_authority')[['treatment', 'treatment_date']].first()
panel_df = panel_df.merge(treatment_map, on='local_authority', how='left')

# Step 3: Compute 'post' indicator
panel_df['financial_year'] = panel_df['financial_year'].astype(int)
panel_df['treatment_year'] = panel_df['treatment_date'].dt.year
panel_df['post'] = (panel_df['financial_year'] >= panel_df['treatment_year']).astype(int)
panel_df.loc[panel_df['treatment'].isna(), 'post'] = 0  # ensure untreated are always 0

classification_vars = [
    'Children_Youth', 'Economic_Social_Development', 'Education_Research',
    'Environment_Animals', 'Grants_Related', 'Health_Disability',
    'Housing_Infrastructure', 'Human_Rights_Advocacy',
    'Other_Miscellaneous', 'Overseas_Famine_Relief', 'Religion_Faith'
]
classification_map = dataset.groupby('local_authority')[classification_vars].max()
panel_df = panel_df.merge(classification_map, on='local_authority', how='left')


In [ ]:
panel_df

In [ ]:
panel_df.dropna(subset=['removed_rate'], inplace=True)

In [ ]:
formula = (
    'removed_rate ~ post + treatment + treatment:post + '
    'C(local_authority) + C(financial_year) + ' +
    ' + '.join(classification_vars)
)

model = smf.ols(formula, data=panel_df).fit(
    cov_type='cluster', cov_kwds={'groups': panel_df['local_authority']}
)
summary = model.summary()
summary

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# --- Assumes dataset is already loaded and 'dataset' is your DataFrame ---

# Ensure date columns are datetime
dataset['date_of_registration'] = pd.to_datetime(dataset['date_of_registration'])
dataset['date_of_removal'] = pd.to_datetime(dataset['date_of_removal'])

# --- Step 1: Create Financial Year End Dates ---
fy_end_dates = (
    dataset['removal_fy'].dropna().astype(int)
)
fy_end_dates = sorted(fy_end_dates[(fy_end_dates >= 2014) & (fy_end_dates <= 2025)].unique())
fy_end_date_map = {fy: pd.Timestamp(f"{int(fy)+1}-03-31") for fy in fy_end_dates}

# --- Step 2: Calculate Active Charities at Each FY End ---
active_by_fy_la = {}
for fy, end_date in fy_end_date_map.items():
    prev_end_date = end_date - pd.DateOffset(years=1)
    active_mask = (
        (dataset['date_of_registration'] <= prev_end_date) &
        (dataset['date_of_removal'].isna() | (dataset['date_of_removal'] > prev_end_date))
    )
    active_count = dataset[active_mask].groupby('local_authority').size().rename(fy)
    active_by_fy_la[fy] = active_count

active_by_fy_la_df = pd.DataFrame(active_by_fy_la).fillna(0).astype(int)

# --- Step 3: Calculate Removals by FY ---
removed_by_fy_la = (
    dataset
    .groupby(['local_authority', 'removal_fy'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Align indices
active_by_fy_la_df = active_by_fy_la_df.reindex(removed_by_fy_la.index)
removal_rate_by_fy_la = removed_by_fy_la / active_by_fy_la_df

# --- Step 4: Convert to Long Format Panel ---
panel_df = removal_rate_by_fy_la.reset_index().melt(
    id_vars='local_authority',
    var_name='financial_year',
    value_name='removed_rate'
)

# --- Step 5: Merge Treatment Info ---
treatment_map = dataset.groupby('local_authority')[['treatment', 'treatment_date']].first()
panel_df = panel_df.merge(treatment_map, on='local_authority', how='left')

panel_df['financial_year'] = panel_df['financial_year'].astype(int)
panel_df['treatment_year'] = panel_df['treatment_date'].dt.year
panel_df['event_time'] = panel_df['financial_year'] - panel_df['treatment_year']

# Create event study dummies with safe names
for i in range(-5, 6):
    if i != -1:  # omit -1 as base
        col_name = f"event_m{abs(i)}" if i < 0 else f"event_{i}"
        panel_df[col_name] = (panel_df['event_time'] == i).astype(int)

# --- Step 6: Add lag of outcome variable ---
panel_df = panel_df.sort_values(['local_authority', 'financial_year'])
panel_df['removed_rate_lag1'] = panel_df.groupby('local_authority')['removed_rate'].shift(1)

# --- Step 7: Add classification controls ---
classification_vars = [
    'Children_Youth', 'Economic_Social_Development', 'Education_Research',
    'Environment_Animals', 'Grants_Related', 'Health_Disability',
    'Housing_Infrastructure', 'Human_Rights_Advocacy',
    'Other_Miscellaneous', 'Overseas_Famine_Relief', 'Religion_Faith'
]
classification_map = dataset.groupby('local_authority')[classification_vars].max()
panel_df = panel_df.merge(classification_map, on='local_authority', how='left')

# --- Step 8: Pre-trend interaction (optional) ---
panel_df['pretrend'] = panel_df['financial_year'] - panel_df['financial_year'].min()
panel_df['pretrend_treat'] = panel_df['pretrend'] * panel_df['treatment']
panel_df_model = panel_df.dropna(subset=['removed_rate', 'removed_rate_lag1'])

# --- Step 9: Build formula ---
event_terms = [f'event_m{abs(i)}' if i < 0 else f'event_{i}' for i in range(-5, 6) if i != -1]

formula = (
    'removed_rate ~ removed_rate_lag1 + pretrend_treat + ' +
    ' + '.join(event_terms) +
    ' + C(local_authority) + C(financial_year) + ' +
    ' + '.join(classification_vars)
)
# --- Step 10: Fit OLS with clustered standard errors ---
model = smf.ols(formula=formula, data=panel_df_model).fit(
    cov_type='cluster', cov_kwds={'groups': panel_df_model['local_authority']}
)
model.summary()

In [ ]:
'''
ADF: remove Non-stationary series (local councils)
'''
from statsmodels.tsa.stattools import adfuller

results = {}
for la in panel_df['local_authority'].unique():
    series = panel_df.loc[panel_df['local_authority'] == la, 'removed_rate'].dropna()
    if len(series) > 10:  # require enough data points
        result = adfuller(series)
        results[la] = {'ADF Statistic': result[0], 'p-value': result[1]}
        
for la, res in results.items():
    print(f"Local Authority: {la}, ADF Stat: {res['ADF Statistic']:.4f}, p-value: {res['p-value']:.4f}")

import pandas as pd

# Assuming your dictionary is stored in variable `results`
# Flatten the dictionary into a list of rows for DataFrame construction
rows = []
for la, stats in results.items():
    rows.append({
        'local_authority': la,
        'adf_stat': stats['ADF Statistic'],
        'p_value': stats['p-value']
    })

df_adf = pd.DataFrame(rows)

# Define thresholds for problematic series
threshold_adf_stat = 1e10

problematic = df_adf[
    (df_adf['p_value'] == 0) |
    (df_adf['p_value'] == 1) |
    (df_adf['adf_stat'].abs() > threshold_adf_stat)
]

print("Problematic Local Authorities:")
print(problematic['local_authority'].tolist())

# Optional: Filter them out from your panel_df if you want clean data
clean_panel_df = panel_df[~panel_df['local_authority'].isin(problematic['local_authority'])]

print(f"Clean panel data now has {clean_panel_df['local_authority'].nunique()} local authorities.")

# Keep only stationary series (p-value < 0.05)
stationary = df_adf[df_adf['p_value'] < 0.05]

# Filter panel data accordingly
stationary_panel_df = panel_df[panel_df['local_authority'].isin(stationary['local_authority'])]

print(f"Panel data now contains {stationary_panel_df['local_authority'].nunique()} stationary local authorities.")


In [ ]:
formula = (
    'removed_rate ~ removed_rate_lag1 + pretrend_treat + ' +
    ' + '.join(event_terms) +
    ' + C(local_authority) + C(financial_year) + ' +
    ' + '.join(classification_vars)
)
# --- Step 10: Fit OLS with clustered standard errors ---
model = smf.ols(formula=formula, data=panel_df_model).fit(
    cov_type='cluster', cov_kwds={'groups': panel_df_model['local_authority']}
)
model.summary()

In [ ]:
# Filter event leads (negative events)
pre_treatment_events = [col for col in panel_df.columns if col.startswith('event_m')]

# Extract their coefficients and p-values from your model
coef_table = model.summary2().tables[1]

In [ ]:
''' 
Check Parallel Trends (Pre-treatment event coefficients)
Treatment Effect Timing (Post-treatment event coefficients)
'''

# Filter event leads (negative events)
pre_treatment_events = [col for col in panel_df.columns if col.startswith('event_m')]

# Extract their coefficients and p-values from your model
coef_table = model.summary2().tables[1]

for ev in pre_treatment_events:
    coef = coef_table.loc[ev, 'Coef.']
    pval = coef_table.loc[ev, 'P>|z|']
    print(f"{ev}: Coef = {coef:.4f}, p-value = {pval:.3f}")


In [ ]:
'''
Coefficients on Classification Groups
'''
classification_vars = [
    'Children_Youth', 'Economic_Social_Development', 'Education_Research',
    'Environment_Animals', 'Grants_Related', 'Health_Disability',
    'Housing_Infrastructure', 'Human_Rights_Advocacy',
    'Other_Miscellaneous', 'Overseas_Famine_Relief', 'Religion_Faith'
]

for var in classification_vars:
    coef = coef_table.loc[var, 'Coef.']
    pval = coef_table.loc[var, 'P>|z|']
    print(f"{var}: Coef = {coef:.4f}, p-value = {pval:.3f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Residuals vs fitted values plot
fitted_vals = model.fittedvalues
residuals = model.resid

from statsmodels.stats.stattools import durbin_watson
dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson statistic: {dw_stat:.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Extract event coefficients and std errors from your model summary
coef_table = model.summary2().tables[1]

# Select event terms (both leads and lags)
event_terms = [col for col in coef_table.index if col.startswith('event')]

# Extract coefficients and standard errors
coefs = coef_table.loc[event_terms, 'Coef.']
std_errs = coef_table.loc[event_terms, 'Std.Err.']

# Calculate 95% confidence intervals
ci_upper = coefs + 1.96 * std_errs
ci_lower = coefs - 1.96 * std_errs

# Convert event names to numeric x-axis values
def event_to_num(ev):
    if ev.startswith('event_m'):
        return -int(ev.split('_m')[1])
    else:
        return int(ev.split('_')[1])

x_vals = [event_to_num(ev) for ev in event_terms]

# Sort by x-axis for plotting
sorted_indices = sorted(range(len(x_vals)), key=lambda i: x_vals[i])
x_sorted = [x_vals[i] for i in sorted_indices]
coefs_sorted = [coefs[i] for i in sorted_indices]
ci_upper_sorted = [ci_upper[i] for i in sorted_indices]
ci_lower_sorted = [ci_lower[i] for i in sorted_indices]

# Plot
plt.figure(figsize=(10,6))
plt.plot(x_sorted, coefs_sorted, marker='o', label='Coefficient')
plt.fill_between(x_sorted, ci_lower_sorted, ci_upper_sorted, color='blue', alpha=0.2, label='95% CI')
plt.axhline(0, color='red', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')
plt.xlabel('Event Time (Years relative to treatment)')
plt.ylabel('Coefficient')
plt.title('Event Study: Lead and Lag Coefficients')
plt.legend()
plt.show()


DRAFT

In [ ]:
# Step 1: Calculate total 2024 housing stock per council (Local authority only)
stock_2024 = (
    housing_stock_df
    .groupby(['Authority_code', 'Authority_name'], as_index=False)['LA_number']
    .sum()
    .rename(columns={'LA_number': 'Stock_2024'})
)

# Step 2: Prepare Net Supply Data
net = housing_net_supply_df.copy()
net = net[net['Year'].astype(str).str.match(r'^\d{4}/\d{2}$')]  # Keep only year-formatted rows
net['Year'] = net['Year'].str[:4].astype(int)  # Convert '2015/16' → 2015

# Pivot to Local Authority x Year format
net_pivot = net.pivot(index='LA_code', columns='Year', values='Net_additions').fillna(0)
net_pivot = net_pivot[sorted(net_pivot.columns, reverse=True)]  # descending order

# Step 3: Merge with 2024 stock
stock = stock_2024.set_index('Authority_code')
net_pivot = net_pivot.reindex(stock.index)  # align index

# Step 4: Calculate total stock backwards from 2024
stock_by_year = pd.DataFrame(index=net_pivot.index)
stock_by_year[2024] = stock['Stock_2024']

for year in sorted(net_pivot.columns, reverse=True):
    if year < 2024:
        stock_by_year[year] = stock_by_year[year + 1] - net_pivot[year]

# Step 5: Merge back Authority name and reshape
stock_by_year = stock_by_year.merge(stock[['Authority_name']], left_index=True, right_index=True)
stock_by_year_reset = stock_by_year.reset_index().melt(id_vars=['Authority_code', 'Authority_name'], 
                                                       var_name='Year', value_name='Housing stock')

# Final pivot
stock_by_year_pivot = stock_by_year_reset.pivot(index=['Authority_code', 'Authority_name'], 
                                                 columns='Year', values='Housing stock').sort_index(axis=1)

# Reset index, rename, and set it again
stock_by_year_pivot = stock_by_year_pivot.reset_index().rename(columns={'Authority_code': 'ONS code'})
stock_by_year_pivot = stock_by_year_pivot.set_index(['ONS code', 'Authority_name'])


In [ ]:
# Filter: Remove NaN and keep only rows where 'Year' matches 'YYYY/YY'
ladataset_filtered = ladataset[
    ladataset['Year'].notna() & ladataset['Year'].str.match(r'^\d{4}/\d{2}$')
]
ladataset_filtered['Year'] = ladataset_filtered['Year'].str[:4].astype(int)

# Pivot
ladataset_pivot = ladataset_filtered.pivot_table(
    index=['ONS code', 'Local authority', 'Measure'],
    columns='Year',
    values='£ millions, cash terms',
    aggfunc='sum'
).reset_index()

# Group by 'con_code' only
population_summary = (
    population
    .groupby('con_code', as_index=False)
    .agg({'con_name': 'first', 'con_number': 'sum'})
    .rename(columns={'con_name': 'Local authority', 'con_number': 'Total Population'})
)

# Display the result
print(population_summary.head())


In [ ]:
# Filter for Core Spending Power (CSP)
csp = ladataset_pivot.copy()

# Keep only relevant years
csp = csp.set_index(['ONS code', 'Local authority']).drop(columns='Measure')

# Calculate % change from 2015
csp['financial_distress'] = (csp[2024] - csp[2015]) / csp[2015]

fd = csp[['financial_distress']].reset_index()

fd.columns.name = None
housing_stock_processed = housing_stock_df[housing_stock_df['Tenure'] == 'Local authority'][['Authority_code', 'LA_number']]
fd_new = fd.merge(housing_stock_processed, left_on='ONS code', right_on='Authority_code', how='inner')
fd_new = fd_new.drop(columns='Authority_code')
fd_new = fd_new.rename(columns={'LA_number': 'LA Housing stock 2024'})


In [ ]:
# Step 1: Normalise both variables
fd_new['financial_distress_norm'] = (
    (fd_new['financial_distress'] - fd_new['financial_distress'].min()) /
    (fd_new['financial_distress'].max() - fd_new['financial_distress'].min())
)

fd_new['LA Housing stock 2024_norm'] = (
    (fd_new['LA Housing stock 2024'] - fd_new['LA Housing stock 2024'].min()) /
    (fd_new['LA Housing stock 2024'].max() - fd_new['LA Housing stock 2024'].min())
)

# Step 2: Avoid division by zero and calculate treatment
fd_new['treatment'] = np.where(
    fd_new['LA Housing stock 2024_norm'] != 0,
    fd_new['financial_distress_norm'] / fd_new['LA Housing stock 2024_norm'],
    0
)
fd_new.rename(columns={'Local authority': 'local_authority'}, inplace=True)

In [ ]:
# Step 2: Filter valid rows
removed = dataset[dataset['removal_fy'].notnull() & dataset['local_authority'].notnull()].copy()
removed['removal_fy'] = removed['removal_fy'].astype(int)

# Step 3: Group by financial year and local authority
removed_by_fy_la = (
    removed
    .groupby(['local_authority', 'removal_fy'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Step 4: Keep only FY 2015–2024
fy_years = list(range(2015, 2025))
removed_by_fy_la = removed_by_fy_la[removed_by_fy_la.columns.intersection(fy_years)]

# Step 5: Total and sort
removed_by_fy_la['Total'] = removed_by_fy_la.sum(axis=1)
removed_by_fy_la = removed_by_fy_la.sort_values(by='Total', ascending=False)

# Step 6: Preview top 10
print("Removed Charities per Financial Year per Local Authority (FY 2015–2024):")
print(removed_by_fy_la.head(10))

In [ ]:
# Step 1: Define year range
years = list(range(2015, 2025))

# Step 2: Calculate percentage change across years
removed_pct_change = removed_by_fy_la[years].pct_change(axis=1) * 100

# Step 3: Round for readability
removed_pct_change = removed_pct_change.round(2)
removed_pct_change